**Time Series Analysis and Forecasting Using ARIMA**

**What is a Time Series Problem:**

In the field of machine learning and data science, most of the real life problems are based on upon the prediction of future
which is totally oblivious to us such as stock market prediction, future sales prediction and so on. Time series problem is 
basically the prediction of such problems using variuos machine learning tools. Time series problem is tackled efficently when first it is analyzed properly and according to that observation suitable algorithm is used.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/portland-oregon-avg-rider-monthly-data/portland-oregon-average-monthly-.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns=["month","average_monthly_ridership"]

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['average_monthly_ridership'].unique()

we can see above anamolous data, which is the lastone.

In [ ]:
df=df.drop(df.index[df['average_monthly_ridership']==' n=114'])

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df.dtypes

* Assign int into average_monthly_ridership column
* assign datetime to month column

In [ ]:
df['average_monthly_ridership'] = df['average_monthly_ridership'].astype(np.int32)

In [ ]:
df.dtypes

In [ ]:
df['month'] = pd.to_datetime(df['month'])

In [ ]:
df.dtypes

**Time Series Analysis:**

terminology which is frequently used in this problem domain.

**Trend** : as a name suggests trend depicts the variation in the output as time increases. it is often nonlinear. sometimes
we will refer to trend as "changing direction" when it might go from an increasing trend to decreasing trend.

**Level**: it is basically depicts baseline value for the time series.

**Seasonal**: As its name depicts it shows the repeated pattern over time. in layman terms, it shows the seasonal variation of data 
over time.

**Noise** : it basically external noises that vary the data randomly.


In [ ]:
# Normal line plot so we can see data variation 
df.plot.line(x='month',y='average_monthly_ridership')

* we can observe that average number of riders is increasing most of time

**Plotting Monthly Variation of Dataset**

it gives us idea about seasonal variation of our dataset

In [ ]:
df1=df

In [ ]:
#only store month
mon = df1['month']

In [ ]:
print(mon.shape)
print(mon.head(2))

In [ ]:
temp=pd.DatetimeIndex(mon)

In [ ]:
temp.dtype

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html

Represented internally as int64, and which can be boxed to Timestamp objects that are subclasses of datetime and carry metadata.

In [ ]:
temp.month

In [ ]:
temp.year

In [ ]:
month = pd.Series(temp.month)

In [ ]:
print(month.dtype)
print(month.head(2))

In [ ]:
df1.columns

In [ ]:
df1=df1.drop(['month'],axis=1)

In [ ]:
df1.head(5)

In [ ]:
df1=df1.join(month)

In [ ]:
df1.head(5)

In [ ]:
sns.barplot(x='month',y='average_monthly_ridership',data=df1)
plt.show()

Well this looks tough to decode. Not a typical box plot. one can infer that data is too sparse for this graph to represent any
pattern. hence it cannot represents monthly variation effectively. we can 

In [ ]:
df1.plot.scatter(x='month',y='average_monthly_ridership')
plt.show()

we can see here the yearly variation of data in this plot. to understand this curve more effectively first look at the 
every row from bottom to top and see each year's variation. to understand yearly variation take a look at each column 
representing a month.


In [ ]:
df2=df[['average_monthly_ridership']]

In [ ]:
df2.head(2)

In [ ]:
df2.dtypes

### Trend Analysis

In [ ]:
df2.rolling(6).mean().plot(figsize=(20,10),linewidth=5,fontsize=20)

rolling window:
choosing rollowing window size, the number of consecutive observations per rolling window. the size of the rolling window will
depend on the sample size, time , and periodicity of the data. in general, you can use a shorting rolling window size for data 
collect in short intervals, and a larger size for data collected in longer intervals. longer rolling window sizes tend to yield 
smoother rolling window eastimates than shorter sizes.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.rolling.html

For trend analysis, we use smoothing techniques. in stastics smoothing a dataset means to create an approximating function 
that attempts to capture important patterens in the data, while leaving out noise or other fine scale structures phenomena. in 
smoothing, the data points of a signal are modified so individual points are reduced, and points that are lower than the adjacent 
points ae increased leading to a smoother signal. we implement smoothing by taking moving averages. Exponential Smoothing average is frequently used to compute smoothed function. here i used the rolling method which is inbuilt in pandas and freuently used for smoothing.

## **Seasonability Analysis**

Two most famous seasonality analysis algorithms are:

* Discrete difference of object
* Periodicity and Autocorrelation 

## Discrete Difference of Object

In [ ]:
df2.diff(periods =4).plot(figsize=(20,10), linewidth=5,fontsize=20)
plt.show()

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html

## Periodicity and Autocorrelation

Auto correlation is the most famous way to understand seasonal variation till now. we can calculate the correlation for 
time series observations with observations with time steps, called lags. because the correlation of the time series observation is
calculated with values of the same series at previous times, this is called a serial correlation or auto correlation.

harizontal axis represents time lags 

In [ ]:
pd.plotting.autocorrelation_plot(df['average_monthly_ridership'])
plt.show()

In [ ]:
pd.plotting.lag_plot(df['average_monthly_ridership'])
plt.show()

The above curve represents the relation between current time stepp and its previous time step

In [ ]:
df = df.set_index('month')

In [ ]:
# Applying Seasonal ARIMA model to forcast the data 
import statsmodels.api as sm
mod = sm.tsa.SARIMAX(df['average_monthly_ridership'], trend='n', order=(0,1,0), seasonal_order=(1,1,1,12))
results = mod.fit()
print(results.summary())

**To check your code**

In [ ]:
df['forecast'] = results.predict(start = 102, end= 120, dynamic= True)  
df[['average_monthly_ridership', 'forecast']].plot(figsize=(12, 8))
plt.show()

**To generate future forcasts**

In [ ]:
def forcasting_future_months(df, no_of_months):
    df_perdict = df.reset_index()
    mon = df_perdict['month']
    mon = mon + pd.DateOffset(months = no_of_months)
    future_dates = mon[-no_of_months -1:]
    df_perdict = df_perdict.set_index('month')
    future = pd.DataFrame(index=future_dates, columns= df_perdict.columns)
    df_perdict = pd.concat([df_perdict, future])
    df_perdict['forecast'] = results.predict(start = 114, end = 125, dynamic= True)  
    df_perdict[['average_monthly_ridership', 'forecast']].iloc[-no_of_months - 12:].plot(figsize=(12, 8))
    plt.show()
    return df_perdict[-no_of_months:]

In [ ]:
predicted = forcasting_future_months(df,10)

In [ ]:
df.tail()